# Test build model

In [ ]:
import tensorflow as tf
from keras import layers
from cnn_classifier import *

In [ ]:
input_shape = (128,128,3)
in_channels = 3
out_classes = 4
filters = [8,8,16,16,32,32,48,64,80]
kernel_size = [3,3,3,3,3,3,3,3,3]
strides = [1,2,1,2,1,2,1,1,1]
padding='valid'
activation='relu'
downsampling_mode='max'
flatten=False
fc=[128,16]
dropout = 0.5
spatial_dropout = 0.2
lambda_l2 = 0.0005
lrate = 0.001
loss = 'sparse_categorical_crossentropy'
metrics = None

In [ ]:
model = create_cnn_classifier_network(input_shape, in_channels, out_classes, filters, kernel_size, strides, padding=padding, activation=activation, downsampling_mode=downsampling_mode, flatten=flatten, fc=fc)
model.summary()

In [ ]:
x = tf.random.normal((1,128,128,3))
y = model(x)
print(y)

# Figures 1 & 2

In [ ]:
import os
import fnmatch
import pickle
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def read_all_rotations(dirname, filebase):
    '''Read results from dirname from files matching filebase'''
    
    # The set of files in the directory
    files = fnmatch.filter(os.listdir(dirname), filebase)
    files.sort()
    results = []
    
    # Loop over matching files
    for f in files:
        fp = open("%s/%s"%(dirname,f), "rb")
        r = pickle.load(fp)
        fp.close()
        results.append(r)
    return results

In [ ]:
filebase = "image_Csize_3_3_3_3_3_Cfilters_10_20_30_40_50_Pool_2_2_2_2_2_Pad_valid_hidden_50_20_drop_0.500_L2_0.000500_LR_0.001000_ntrain_*_rot_*_results.pkl"
results = read_all_rotations("results", filebase)
filebase = "image_Csize_3_3_3_3_3_3_3_3_3_Cfilters_8_8_16_16_32_32_48_64_80_Pool_1_2_1_2_1_2_1_1_1_Pad_valid_hidden_128_16_drop_0.500_LR_0.001000_ntrain_*_rot_*_results.pkl"
results_d = read_all_rotations("results", filebase)

In [ ]:
print(len(results)) # should be 20*7 = 140 eventually
results[0].keys()

In [ ]:
results[0]['history'].keys()

In [ ]:
# plot learning curve
plt.figure()
plt.plot(results[0]['history']['val_loss'], label='loss_val 0')
plt.plot(results[1]['history']['val_loss'], label='loss_val 1')
plt.plot(results[2]['history']['val_loss'], label='loss_val 2')
plt.plot(results[3]['history']['val_loss'], label='loss_val 3')
plt.plot(results[4]['history']['val_loss'], label='loss_val 4')
plt.ylabel('validation loss (sparse categorical cross-entropy)')
plt.xlabel('epochs')
plt.legend()

In [ ]:
# plot learning curve
plt.figure()
plt.plot(results[0]['history']['val_sparse_categorical_accuracy'], label='acc_val 0')
plt.plot(results[1]['history']['val_sparse_categorical_accuracy'], label='acc_val 1')
plt.plot(results[2]['history']['val_sparse_categorical_accuracy'], label='acc_val 2')
plt.plot(results[3]['history']['val_sparse_categorical_accuracy'], label='acc_val 3')
plt.plot(results[4]['history']['val_sparse_categorical_accuracy'], label='acc_val 4')
plt.ylabel('validation accuracy (sparse categorical accuracy)')
plt.xlabel('epochs')
plt.legend()

In [ ]:
test_eval_shallow = []
test_eval_deep = []
for i in range(5):
    test_eval_shallow.append(results[i]['predict_testing_eval'][1])
    test_eval_deep.append(results_d[i]['predict_testing_eval'][1])
test_eval_shallow

In [ ]:
plt.figure()
plt.hist(test_eval_shallow, alpha = 0.5)
plt.hist(test_eval_deep, alpha = 0.5)

In [ ]:
from core50 import *

rotation = 0
base_dir = '/home/ikang/datasets/core50_128x128'

core = Core50('core50_df.pkl')

# Set the problem class IDs
objects = [4,5,6,8]
core.set_problem_class_by_equality('class', objects)

# Select only these object classes (remove all others)
core.filter_problem_class()

# Folds by pairs of condition ((1,2), (3,4), ...)
folds = core.create_subsets_by_membership('condition', list(zip(range(1,11,2),range(2,11,2))))

_, _, df_testing = core.create_training_validation_testing(rotation, folds, 3)

# use with mini dataset
df_testing = df_testing[df_testing['fname'].str.contains('0.png')]

ds_testing = tf.data.Dataset.from_tensor_slices(df_testing[['fname', 'problem_class']].to_numpy())

ds_testing = ds_testing.map(lambda x: tf.py_function(func=Core50.prepare_single_example, inp=[base_dir, x], Tout=(tf.float32, tf.int8)))

#ds_testing.map(lambda x: x*2)

In [ ]:
for element in ds_testing.take(1):
    plt.imshow(element[0].numpy())
    print('true:', element[1].numpy())